In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [6]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [8]:
X = scaler.fit_transform(X)

In [9]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [10]:
X.shape

(768, 8)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [62]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Input, Dense, Dropout

In [13]:
model = Sequential()

model.add(Input(shape=(8,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.3863 - loss: 0.8200 - val_accuracy: 0.4026 - val_loss: 0.7815
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5213 - loss: 0.7229 - val_accuracy: 0.5584 - val_loss: 0.7012
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6207 - loss: 0.6667 - val_accuracy: 0.6169 - val_loss: 0.6452
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6908 - loss: 0.6155 - val_accuracy: 0.7078 - val_loss: 0.6037
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7534 - loss: 0.5710 - val_accuracy: 0.7273 - val_loss: 0.5745
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7176 - loss: 0.5611 - val_accuracy: 0.7468 - val_loss: 0.5526
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7405 - loss: 0.5343 - val_accuracy: 0.7662 - val_loss: 0.5345
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7696 - loss: 0.5136 - val_accuracy: 0.7662 - 

In [15]:
# 1.How to select appropriate optimizer
# 2.NO, of nodes in a layer
# 3. How to select no. of layers
# 4.All in one model

In [16]:
import keras_tuner as kt

In [17]:
def build_model(hp):
    model = Sequential()
    
    model.add(Input(shape=(8,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = hp.Choice('optimizer', values=['adam','sgd','rmsprop','adadelta'])
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [18]:
tunner = kt.RandomSearch(build_model,
                         objective='val_accuracy',
                         max_trials=5)

In [19]:
tunner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 13s


In [20]:
tunner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [21]:
model = tunner.get_best_models(num_models=1)[0]

c:\Users\harsha s n\anaconda3\envs\advproj\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.7124 - loss: 0.5847 - val_accuracy: 0.7792 - val_loss: 0.5358
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7254 - loss: 0.5556 - val_accuracy: 0.7857 - val_loss: 0.5134
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7367 - loss: 0.5316 - val_accuracy: 0.7792 - val_loss: 0.4996
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7600 - loss: 0.5120 - val_accuracy: 0.7727 - val_loss: 0.4896
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7503 - loss: 0.5082 - val_accuracy: 0.7792 - val_loss: 0.4828
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7517 - loss: 0.5024 - val_accuracy: 0.7792 - val_loss: 0.4767
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7511 - loss: 0.4981 - val_accuracy: 0.7792 - val_loss: 0.4720
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7682 - loss: 0.4825 - val_accuracy: 0.78

In [40]:
def build_model(hp):
    
    model = Sequential()
    
    units = hp.Int('units', min_value=8, max_value=128)
    
    model.add(Input(shape=(8,)))
    model.add(Dense(units=units, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [41]:
tunner = kt.RandomSearch(build_model,
                         objective='val_accuracy',
                         max_trials=5,
                         project_name='Check_Values')

In [42]:
tunner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6363636255264282

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 08s


In [43]:
tunner.get_best_hyperparameters()[0].values

{'units': 90}

In [44]:
def build_model(hp):
    
    model = Sequential()
    
    model.add(Input(shape=(8,)))
    model.add(Dense(72, activation='relu'))
    
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(72, activation='relu'))
        
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])    
    
    return model    

In [45]:
tunner = kt.RandomSearch(build_model,
                         objective='val_accuracy',
                         max_trials=5,
                         project_name='num_layers')

In [46]:
tunner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.8181818127632141

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 14s


In [47]:
tunner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [48]:
model = tunner.get_best_models(num_models=1)[0]

c:\Users\harsha s n\anaconda3\envs\advproj\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [49]:
model.fit(X_train, y_train, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7866 - loss: 0.4528 - val_accuracy: 0.7922 - val_loss: 0.4519
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8083 - loss: 0.4099 - val_accuracy: 0.8052 - val_loss: 0.4535
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7933 - loss: 0.4317 - val_accuracy: 0.8052 - val_loss: 0.4520
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8071 - loss: 0.4013 - val_accuracy: 0.8052 - val_loss: 0.4585
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7706 - loss: 0.4423 - val_accuracy: 0.7987 - val_loss: 0.4630
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8410 - loss: 0.3678 - val_accuracy: 0.8182 - val_loss: 0.4646
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8316 - loss: 0.3816 - val_accuracy: 0.7987 - val_loss: 0.4598
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8163 - loss: 0.3611 - val_accuracy: 0.82

In [63]:
def build_model(hp):
    
    model = Sequential()
    
    counter = 0
    
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        
        if counter == 0:
            
            model.add(
                Dense(
                    hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                    activation= hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),
                    input_dim=8
                )
            )
            model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
        else:
            model.add(
                Dense(
                    hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                    activation= hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])
                )
            )
            model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
        counter += 1
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])   
    return model     

In [64]:
tunner = kt.RandomSearch(build_model,
                         objective='val_accuracy',
                         max_trials=3,
                         project_name='final1')

c:\Users\harsha s n\anaconda3\envs\advproj\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [65]:
tunner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 10s


In [66]:
tunner.get_best_hyperparameters()[0].values

{'num_layers': 8,
 'units0': 16,
 'activation0': 'tanh',
 'dropout0': 0.1,
 'optimizer': 'rmsprop',
 'units1': 16,
 'activation1': 'tanh',
 'dropout1': 0.4,
 'units2': 8,
 'activation2': 'tanh',
 'dropout2': 0.1,
 'units3': 120,
 'activation3': 'tanh',
 'dropout3': 0.1,
 'units4': 104,
 'activation4': 'tanh',
 'dropout4': 0.4,
 'units5': 104,
 'activation5': 'sigmoid',
 'dropout5': 0.4,
 'units6': 8,
 'activation6': 'relu',
 'dropout6': 0.1,
 'units7': 8,
 'activation7': 'relu',
 'dropout7': 0.1}

In [67]:
model = tunner.get_best_models(num_models=1)[0]

c:\Users\harsha s n\anaconda3\envs\advproj\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [68]:
model.fit(X_train, y_train, epochs=200, initial_epoch=5, validation_data=(X_test, y_test))

Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7200 - loss: 0.5744 - val_accuracy: 0.7922 - val_loss: 0.5246
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6989 - loss: 0.5891 - val_accuracy: 0.7857 - val_loss: 0.5339
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6938 - loss: 0.5681 - val_accuracy: 0.7857 - val_loss: 0.5183
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7302 - loss: 0.5657 - val_accuracy: 0.7922 - val_loss: 0.5084
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7316 - loss: 0.5371 - val_accuracy: 0.7857 - val_loss: 0.5082
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7134 - loss: 0.5742 - val_accuracy: 0.8052 - val_loss: 0.4918
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7356 - loss: 0.5299 - val_accuracy: 0.7987 - val_loss: 0.4863
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7225 - loss: 0.5527 - val_accuracy: 0.77